In [142]:
import pandas as pd 
import numpy as np 
from sklearn.metrics import accuracy_score

In [143]:
# Helper function to get classifier name
def get_clf_name(estimator):
    return(estimator.__class__.__name__)

def get_clf_class(estimator):
    return clf.__class__.__module__.split(".")[2]


In [144]:
from sktime.datasets import load_arrow_head
from sktime.classification.dictionary_based import IndividualBOSS
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X, y = load_arrow_head(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y)
classifier = IndividualBOSS()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
accuracy_score(y_test, y_pred)

0.6037735849056604

In [145]:
print(X.shape)

(211, 1)


In [146]:
print(y)

['0' '1' '2' '0' '1' '2' '0' '1' '2' '0' '1' '2' '0' '1' '2' '0' '1' '2'
 '0' '1' '2' '0' '1' '2' '0' '1' '2' '0' '1' '2' '0' '1' '2' '0' '1' '2'
 '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0'
 '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0'
 '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0'
 '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '1' '1' '1'
 '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1'
 '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1'
 '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '2' '2' '2' '2'
 '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2'
 '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2'
 '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2']


# Get Dataset

1. List the names of all the files that will be used in the training  
2. Since the values only have the x_value, specify the number used for classification (eg s1 = 1, s2 = 2)  
3. Itterate over all file_names, create a df 
4. Convert the 2d_array dataframe into a nested df
5. add approiate y value and append it to the df  
4. Concatenate all the dataframes in the list  

In [147]:
from sktime.datatypes._panel._convert import (
    from_2d_array_to_nested,
    from_nested_to_2d_array,
    is_nested_dataframe,
)

In [158]:
file_names = ["s1_no_vol", "s2_no_vol"]

dfs= []
y = []
for file_name in file_names:
    df = pd.read_csv ("../datasets/"+file_name+".csv")
    df.drop(columns=df.columns[:1],axis=1, inplace=True)
    nested_df = from_2d_array_to_nested(df)
    y.extend([file_name[1]] * df.shape[0])
    dfs.append(nested_df)
    
combined_df = pd.concat(dfs)

   0  1  2  3    4    5    6    7    8    9  ...  91  92  93  94  95  96  97  \
0  0  0  0  0    0    0    0    0  100  100  ...   0   0   0   0   0   0   0   
1  0  0  0  0    0    0  100  100  100  100  ...   0   0   0   0   0   0   0   
2  0  0  0  0  100  100  100  100  100  100  ...   0   0   0   0   0   0   0   

   98  99  y  
0   0   0  1  
1   0   0  1  
2   0   0  1  

[3 rows x 101 columns]
     0    1    2    3    4    5    6    7    8    9  ...   91   92   93  94  \
0    0    0    0    0    0    0    0    0    0    0  ...  100  100  100   0   
1  100  100  100  100  100  100  100  100  100  100  ...    0    0    0   0   
2    0    0    0    0    0    0    0    0    0    0  ...    0    0    0   0   

   95  96  97  98  99  y  
0   0   0   0   0   0  1  
1   0   0   0   0   0  1  
2   0   0   0   0   0  1  

[3 rows x 101 columns]


In [154]:
print(combined_df.shape)

(20000, 1)


In [149]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(combined_df[0], y)

## Define the classifiers

Define which classifiers that will be used and store it in a list

In [150]:
from sktime.classification.dictionary_based import IndividualBOSS
from sktime.classification.dictionary_based import ContractableBOSS

clfs = []

clfs.append(ContractableBOSS(n_parameter_samples=25, max_ensemble_size=5))
clfs.append(IndividualBOSS())

In [151]:
types = []

for clf in clfs: 
    types.append(get_clf_class(type))

## Classify the Dataset for every classifier

Classify the dataset with each classifier and save their accuracy score

In [152]:
acc_scores = []

In [153]:
for clf in clfs: 
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    acc_score =accuracy_score(y_test, y_pred)
    acc_scores.append(acc_score)

TypeError: X is not of a supported input data type.X must be in a supported mtype format for Panel, found <class 'pandas.core.series.Series'>Use datatypes.check_is_mtype to check conformance with specifications.

## Sort According to Type


In [ ]:
types, acc_scores, clfs = zip(*sorted(zip(types, acc_scores, clfs)))

## Print the results

Itterate over the classifier and accuracy score and display in a nice "table"

In [ ]:
print(f"{'Type' : <20}{'Name' : <40}{'Accuracy' : <15}")
for type, clf, acc_score in zip(types, clfs, acc_scores):
    acc_score_perc = '{:0.0f}'.format(acc_score*100)
    print(f"{get_clf_class(clf): <20}{get_clf_name(clf) : <40}{(acc_score_perc)+'%':<15}")
